## torch.cuda.amp.autocast
这是一个实现**自动混合精度训练（Automatic Mixed Precision, AMP）**的关键上下文管理器</br>
 AMP 的思想是，在训练过程中，根据操作的类型和敏感度，混合使用 FP16 和 FP32 两种精度以节约显存</br>
 操作类型感知： autocast 在其上下文管理器中执行代码时，会检查每个 PyTorch 操作（例如，矩阵乘法 torch.matmul，卷积 nn.Conv2d，激活函数 nn.ReLU 等）。</br>

动态类型提升 (Dynamic Type Promotion)：</br>

对于那些在 FP16 下效率更高且数值稳定性较好的操作（例如，矩阵乘法、卷积），autocast 会自动将输入张量转换为 FP16，并使用 FP16 执行这些操作。</br>
对于那些在 FP16 下可能存在数值不稳定（如 Softmax、LogSoftmax、BN 层的统计计算等）或精度要求较高的操作，autocast 会自动将输入提升回 FP32 执行这些操作，以避免精度损失。</br>
这种判断和转换是在运行时动态发生的，由 PyTorch 内部的逻辑管理。</br>
梯度计算：</br>
前向传播（forward pass）中的计算结果（激活值）可能存储为 FP16。</br>
在反向传播（backward pass）时，梯度也需要计算。为了确保数值稳定性，PyTorch AMP 通常会使用一个叫做 torch.cuda.amp.GradScaler 的组件。</br>
## GradScaler 的作用是进行损失缩放（Loss Scaling）。</br>
由于 FP16 的数值范围较小，如果梯度值太小，它们可能会下溢为零，导致参数无法更新。</br>GradScaler 会在计算损失后，将其乘以一个大的缩放因子，使得梯度值变大，避免下溢。在反向传播计算完成后，梯度在传回 FP32 参数之前，会被反向缩放，恢复其真实大小。

In [ ]:
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader

# 1. 定义一个简单的Dataset
class SimpleDataset(Dataset):
    def __init__(self, num_samples=1000, input_dim=10, output_dim=1):
        self.num_samples = num_samples
        self.input_dim = input_dim
        self.output_dim = output_dim
        # 生成随机输入数据和对应的线性目标
        self.X = torch.randn(num_samples, input_dim)
        self.y = torch.randn(num_samples, output_dim) * 5 + 10 # 简单模拟一些变化

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# 1. 自定义一个简单的演示模型
class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(10, 20)
    self.relu = nn.ReLU()
    self.layer2 = nn.Linear(20, 1)

  def forward(self, x):
    x = self.layer1(x)
    x = self.relu(x)
    x = self.layer2(x)
    return x

In [ ]:
model = MyModel().cuda()
# 2. 实例化一个优化器
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
# 3. 实例化一个损失函数
criterion = nn.MSELoss()
# 4. 实例化一个GradScaler，进行梯度缩放，预防因为FP16导致梯度消失
scaler = GradScaler()

/tmp/ipython-input-3-2770131166.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
def main():
  for epoch in range(11451):
    for batch_idx, (data, target) in enumerate(dataloader):
      data, target = data.cuda(), target.cuda()
      optim.zero_grad()
      # 梯度归零

      # 5. 使用上下文管理器autocast
      with autocast():
        output = model(data)
        loss = criterion(output, target)
      # 6. 使用scaler进行梯度缩放
      scaler.scale(loss).backward()
      # 它会检查梯度是否有效（非Inf和Nan），然后缩放梯度并更新参数
      scaler.step(optim)

      # 7. 缩放因子，假如在上面没有无效梯度，则缩放因子会稍微增加
      scaler.update()

      if batch_idx % 100 == 0:
        print(f"Epoch{epoch}, Batch{batch_idx}, Loss:{loss.item():.4f}")


In [ ]:
input_dim = 10
output_dim = 1
num_samples = 10000 # 更多数据
batch_size = 64
num_epochs = 10 # 增加训练周期

dataset = SimpleDataset(num_samples=num_samples, input_dim=input_dim, output_dim=output_dim)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
main()